В этом задании будет использоваться датасет digits из sklearn.datasets. Оставьте последние 25% объектов для контроля качества, разделив X и y на X_train, y_train и X_test, y_test.

Целью задания будет реализовать самый простой метрический классификатор — метод ближайшего соседа, а также сравнить качество работы реализованного вами 1NN с RandomForestClassifier из sklearn на 1000 деревьях.

In [1]:
import warnings

warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd

from sklearn.datasets import load_digits
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

In [3]:
def write_answer(filename, answer):
    with open(filename, "w") as fout:
        fout.write(str(answer))

In [4]:
digits = load_digits()

In [5]:
X, y = digits.data, digits.target

X_train, X_test = X[:int(X.shape[0]*.75)], X[int(X.shape[0]*.75):]
y_train, y_test = y[:int(len(y)*.75)], y[int(len(y)*.75):]

In [6]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((1347, 64), (450, 64), (1347,), (450,))

## Задание 1

Реализуйте самостоятельно метод одного ближайшего соседа с евклидовой метрикой для задачи классификации. Можно не извлекать корень из суммы квадратов отклонений, т.к. корень — монотонное преобразование и не влияет на результат работы алгоритма.

Никакой дополнительной работы с признаками в этом задании делать не нужно — мы еще успеем этим заняться в других курсах. Ваша реализация может быть устроена следующим образом: можно для каждого классифицируемого объекта составлять список пар (расстояние до точки из обучающей выборки, метка класса в этой точке), затем сортировать этот список (по умолчанию сортировка будет сначала по первому элементу пары, затем по второму), а затем брать первый элемент (с наименьшим расстоянием).

Сортировка массива длиной N требует порядка N log N сравнений (строже говоря, она работает за O(N log N)). Подумайте, как можно легко улучшить получившееся время работы. Кроме простого способа найти ближайший объект всего за N сравнений, можно попробовать придумать, как разбить пространство признаков на части и сделать структуру данных, которая позволит быстро искать соседей каждой точки. За выбор метода поиска ближайших соседей в KNeighborsClassifier из sklearn отвечает параметр algorithm — если у вас уже есть некоторый бэкграунд в алгоритмах и структурах данных, вам может быть интересно познакомиться со структурами данных ball tree и kd tree.

Доля ошибок, допускаемых 1NN на тестовой выборке, — ответ в задании 1.

In [7]:
from scipy.spatial import distance

In [15]:
y_predict_kNN = []

for test_value in X_test:
    idx_min_dist, min_dist = 0, distance.euclidean(test_value, X_train[0])
    for train_idx, train_value in enumerate(X_train):
        train_min_dist = distance.euclidean(test_value, train_value)
        if train_min_dist < min_dist: idx_min_dist, min_dist = train_idx, train_min_dist
    y_predict_kNN.append(y_train[idx_min_dist])

In [18]:
score = accuracy_score(y_test, y_predict_kNN)
write_answer('answer_1.txt', 1 - score)

## Задание 2

Теперь обучите на обучающей выборке RandomForestClassifier(n_estimators=1000) из sklearn. Сделайте прогнозы на тестовой выборке и оцените долю ошибок классификации на ней. Эта доля — ответ в задании 2. Обратите внимание на то, как соотносится качество работы случайного леса с качеством работы, пожалуй, одного из самых простых методов — 1NN. Такое различие — особенность данного датасета, но нужно всегда помнить, что такая ситуация тоже может иметь место, и не забывать про простые методы.

In [10]:
rf = RandomForestClassifier(n_estimators=1000)
rf.fit(X_train, y_train)
y_predict = rf.predict(X_test)
score = accuracy_score(y_test, y_predict)

write_answer('answer_2.txt', 1 - score)